In [ ]:
import  scipy.io as sio
import os
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import glob
import random
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.utils import to_categorical


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


# attentonlayer

In [ ]:

from keras.layers import Layer
import keras.backend as K


class SpatialAttention(Layer):
    def __init__(self,
                 **kwargs):
        super(SpatialAttention, self).__init__(**kwargs)

    def build(self, input_shape):
        self.kernel = self.add_weight(shape=[input_shape[-2], 1],
                                 name='kernel',
                                 initializer='ones',
                                 trainable=True)

        self.bias = self.add_weight(shape=[input_shape[-2]],
                                 name='bias',
                                 initializer='zeros',
                                 trainable=True)

    def call(self, inputs, training=None):

        input_shape = K.int_shape(inputs)
        mid = input_shape[-2] // 2

        coe = K.l2_normalize(K.batch_dot(inputs, K.permute_dimensions(inputs, pattern=(0, 2, 1))), axis=-1)
        coe0 = K.expand_dims(coe[:, mid, :], axis=-1) * self.kernel
        w = K.batch_dot(coe, coe0) + K.expand_dims(self.bias, axis=-1)
        outputs = K.softmax(w, axis=-2) * inputs

        return outputs

    def get_config(self):
        config = {}
        base_config = super(SpatialAttention, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

    def compute_output_shape(self, input_shape):
        return tuple(input_shape)

# second pool layer

In [ ]:

from keras.layers import Layer
import keras.backend as K


class SecondOrderPooling(Layer):
    def __init__(self,
                 **kwargs):
        super(SecondOrderPooling, self).__init__(**kwargs)

    def call(self, inputs, training=None):
        input_shape = K.int_shape(inputs)

        outputs = K.batch_dot(K.permute_dimensions(inputs, pattern=(0, 2, 1)), inputs, axes=[2, 1])
        outputs = K.reshape(outputs, [-1, input_shape[2] * input_shape[2]])

        return outputs

    def get_config(self):
        config = {}
        base_config = super(SecondOrderPooling, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

    def compute_output_shape(self, input_shape):
        if len(input_shape) == 4:
            output_shape = list([None, input_shape[1], input_shape[3] * input_shape[3]])
        else:
            output_shape = list([None, input_shape[2] * input_shape[2]])
        return tuple(output_shape)

# model

In [ ]:
from keras.layers import Input
from keras.layers import Flatten, Dense, Reshape,BatchNormalization,Dropout,Lambda
import math
from tensorflow.keras.models import Model
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda, Activation
from keras.layers import ZeroPadding2D, ZeroPadding1D
from keras import backend as K

In [ ]:
import numpy
def Symmetry(a, rtol=1e-05):
    return numpy.allclose( a.T, rtol=rtol)

In [ ]:
def aspn(img_rows, img_cols, num_PC, nb_classes):
    CNNInput = Input(shape=(img_rows, img_cols, num_PC), name='i0')
    print(CNNInput.shape)
    F = Reshape([img_rows * img_cols, num_PC])(CNNInput)
    F = BatchNormalization()(F)
    F = Dropout(rate=0.5)(F)

    F = Lambda(lambda x: K.l2_normalize(x, axis=-1), name='f2')(F)
    F = SpatialAttention(name='f3')(F)
    F = SecondOrderPooling(name='feature1')(F)
    F = Lambda(lambda x: K.l2_normalize(x, axis=-1), name='feature2')(F)
    print(F.shape)
    R1 = Reshape((32,32,1))(F)
    print(R1.shape)
    upsamp_1 = UpSampling2D(size=(2,2))(R1)
    print(upsamp_1.shape)
    upsamp_2 = UpSampling2D(size=(2,2))(upsamp_1)
    print(upsamp_2.shape)
    upsamp_3 = UpSampling2D(size=(2,2))(upsamp_2)
    print(upsamp_3.shape)
    #upsamp_1 = ZeroPadding2D(padding=((2,2),(2,2)))(upsamp_3)
    #upsamp_2= ZeroPadding2D(padding=((2,2),(2,2)))(upsamp_1)
    #upsamp_3 = ZeroPadding2D(padding=((2,2),(2,2)))(upsamp_2)
    #upsamp_4= ZeroPadding2D(padding=((2,2),(2,2)))(upsamp_3)
    
    #print(upsamp_1.shape)
    conv = Conv2D(41, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(upsamp_3)
    print(conv.shape)
    outputs = Conv2D(nb_classes, (1, 1), activation='softmax')(conv)
     
   #F = Dense(nb_classes, activation='softmax', name='classifier', kernel_initializer ='he_normal')(F)
    
    model = Model(inputs=[CNNInput], outputs=outputs)
    model.summary()
    return model

#aspn(256 ,256, 32, 41)


In [ ]:
def load_img(img_dir, img_list):
   images = []
   ground_truth_list = []

   for i, image_name in enumerate(img_list):

      if (image_name.split('.')[1] == 'npz'):
         image_file_path = os.path.join(img_dir, image_name)

         MRI_image = np.load(image_file_path)

         image = MRI_image['a']

         # image = np.transpose(MRI_image['a'],(1,2,0))          No need to transpose

         # images.append(image[56:184, 56:184])

         images.append(image)

         image_ground_truth = MRI_image['b'].astype(np.uint8)

         # image_ground_truth = image_ground_truth[56:184, 56:184]

         # n_classes = len(np.unique(image_ground_truth))           not all images have all 10 classes labels so only going to take 10 as total classes
         #print(np.unique(image_ground_truth))
         n_classes = 41

         image_ground_truth = to_categorical(image_ground_truth, num_classes=n_classes)

         ground_truth_list.append(image_ground_truth)

         pass

      pass

   images = np.array(images)
   ground_truth_list = np.array(ground_truth_list)

   return (images, ground_truth_list)


def imageLoader(img_dir, img_list, batch_size):
   L = len(img_list)

   # keras needs the generator infinite, so we will use while true
   while True:

      batch_start = 0
      batch_end = batch_size

      while batch_start < L:
         limit = min(batch_end, L)

         X, Y = load_img(img_dir, img_list[batch_start:limit])

         yield (X, Y)  # a tuple with two numpy arrays with batch_size samples

         batch_start += batch_size
         batch_end += batch_size


In [ ]:
import random
def load_npz_files(npz_path):
      train_files = []
      train_files.append([name for name in glob.glob(npz_path + "*.npz")])
      train_files = train_files[0]
      random.shuffle(train_files)
      return train_files

In [ ]:
from sklearn.decomposition import FactorAnalysis
def applyFA(X, numComponents=32):
    print('FA starts')
    newX = np.reshape(X, (-1, X.shape[2]))
    fa = FactorAnalysis(n_components=numComponents, random_state=0)
    newX = fa.fit_transform(newX)
    newX = np.reshape(newX, (X.shape[0], X.shape[1], numComponents))
    return newX, fa



In [ ]:
import random
import numpy as np
from sklearn.model_selection import train_test_split
if __name__ == "__main__":
   #split data 
   selected_fles=[]
   npz_files_folder_path = '/content/drive/MyDrive/Aiman/HSI/ip+salinas+paviau/'
   all_fles =os.listdir(npz_files_folder_path)
   print('all fles' ,len(all_fles))
   selected_fles.append(all_fles[0:22])
   print('selected fles' ,len(selected_fles))
   X_trn, X_tst= train_test_split(all_fles, test_size=0.2, random_state=42)
   
   npz_files_folder_path = '/content/drive/MyDrive/Aiman/HSI/ip+salinas+paviau/'
   batch_size = 1
   train_images_dataGenerator = imageLoader(img_dir=npz_files_folder_path, img_list=X_trn,batch_size=batch_size)
  
   scaler = MinMaxScaler()
   # Update batch size to vary image batches

   validation_batch_size = 1

   training_npz_folder_path = '/content/drive/MyDrive/Aiman/HSI/ip+salinas+paviau/'

   training_npz_files_list = X_trn[0:1]
   print(len(X_trn))
   print(len(X_tst))
   #print(training_npz_files_list)
   validation_npz_folder_path = '/content/drive/MyDrive/Aiman/HSI/ip+salinas+paviau/'

   validation_npz_files_list = X_tst[2:3]

   print(validation_npz_files_list)

   train_images_dataGenerator = imageLoader(img_dir=training_npz_folder_path, img_list=training_npz_files_list, batch_size=batch_size)



   validation_images_dataGenerator = imageLoader(img_dir=validation_npz_folder_path, img_list=validation_npz_files_list, batch_size=validation_batch_size)
  

   X, Y = train_images_dataGenerator.__next__()
   print(validation_images_dataGenerator)
   print('X' ,X.shape)
   print('Y',Y.shape)
   X= np.squeeze(X ,axis=0)
   K = 32
   X,fa = applyFA(X, numComponents=K)
   print('X' ,X.shape)
   X= np.expand_dims(X ,axis=0)
   print('X' ,X.shape)
   steps_per_epoch = len(training_npz_files_list) // batch_size
   val_steps_per_epoch = len(validation_npz_files_list) // batch_size

   print(steps_per_epoch)

   model = aspn(img_rows=256, img_cols=256, num_PC=32, nb_classes=41)

   # model.compile(optimizer = optim, loss=total_loss, metrics=metrics)

   model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

   print(model.summary())

   print(model.input_shape)
   print(model.output_shape)

   history = model.fit(train_images_dataGenerator,
                       steps_per_epoch=steps_per_epoch,
                       epochs=500,
                       verbose=1,
                       validation_data=validation_images_dataGenerator,
                       validation_steps=val_steps_per_epoch,
                       
                       )

   model.save('/content/drive/MyDrive/Aiman/HSI/dense_unet.hdf5')
   loss = history.history['loss']
   val_loss = history.history['val_loss']
   epochs = range(1, len(loss) + 1)
   plt.plot(epochs, loss, 'y', label='Training loss')
   plt.plot(epochs, val_loss, 'r', label='Validation loss')
   plt.title('Training and validation loss')
   plt.xlabel('Epochs')
   plt.ylabel('Loss')
   plt.legend()
   plt.savefig("loss_curve.pdf")
   plt.savefig("loss_curve.jpg")
   plt.show()

   acc = history.history['accuracy']
   val_acc = history.history['val_accuracy']

   plt.plot(epochs, acc, 'y', label='Training accuracy')
   plt.plot(epochs, val_acc, 'r', label='Validation accuracy')

   plt.title('Training and validation accuracy')
   plt.xlabel('Epochs')
   plt.ylabel('Accuracy')
   plt.legend()
   plt.savefig("acc_curve.pdf")
   plt.savefig("acc_curve.jpg")
   plt.show()

   from keras.models import load_model

npz_files_folder_path = '/content/drive/MyDrive/Aiman/HSI/ip+salinas+paviau/'
hyperspectral_npz_files = load_npz_files(npz_files_folder_path)
batch_size = 4
train_images_dataGenerator = imageLoader(img_dir=npz_files_folder_path, img_list=hyperspectral_npz_files,
                                         batch_size=batch_size)

X, Y = train_images_dataGenerator.__next__()

my_model = load_model('/content/drive/MyDrive/Aiman/HSI/dense_unet.hdf5')

print(my_model)

print()

"""npz_files_folder_path = '/content/drive/MyDrive/pavia_Hyperspectral_images/'

file_name_list = os.listdir(npz_files_folder_path)

X, Y = load_img(img_dir=npz_files_folder_path,img_list=file_name_list)"""

query_hyperspectral_image = X[0]

print(query_hyperspectral_image.shape)

test_img_input = np.expand_dims(query_hyperspectral_image, axis=0)

print(test_img_input.shape)

# Prediction of UNET model

test_prediction = my_model.predict(test_img_input)

print(test_prediction.shape)

# converting prediction to normal from to categorial i.e   (256 x 256 x 10) ==> (256 x 256 x 1)

test_prediction_argmax = np.argmax(test_prediction, axis=3)[0, :, :]

print(test_prediction_argmax.shape)

prediction = test_prediction_argmax

ground_truth = Y[0]

print("-----")

# ground_truth = np.expand_dims(ground_truth, axis=2)

print(ground_truth.shape)

ground_truth = np.argmax(ground_truth, axis=2)

plt.figure(figsize=(12, 8))
plt.subplot(231)
plt.title('Testing Image')
plt.imshow(query_hyperspectral_image[:, :, 1], cmap='gray')
plt.subplot(232)
plt.title('Testing Label')
plt.imshow(ground_truth)
plt.subplot(233)
plt.title('Prediction on test image')
plt.imshow(prediction[:, :])
plt.show()

print(np.unique(prediction))



all fles 256
selected fles 1
204
52
['ip+salinas+paviau79.npz']
<generator object imageLoader at 0x7fe7d61bf150>
X (1, 256, 256, 150)
Y (1, 256, 256, 41)
FA starts
X (256, 256, 32)
X (1, 256, 256, 32)
1
(None, 256, 256, 32)


AttributeError: ignored